In [12]:
import os

import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from plyfile import *
import glob
import time
from kitti360scripts.helpers.project import Camera, CameraPerspective
from kitti360scripts.helpers.ply import *

In [13]:
#Main path

KITTI360Path="/media/ren/EXTERNAL_USB/KITTI360_DATASET/"

print(KITTI360Path)

# Intrinsicpath=os.path.join(KITTI360Path,'calibration/perspective.txt')

# P_rect=CameraPerspective.load_intrinsics(Intrinsicpath)

# print(P_rect)




/media/ren/EXTERNAL_USB/KITTI360_DATASET/


In [14]:
#image pixel 
width=1408 #width
height=376 #height
#template pixel
tempWl=100 #width
tempWr=1300
tempHu=10 #height
tempHd=350
#index of frame
frameId=0
#maximum and minimum value within matrix
max_v=0
min_v=0
# cam2pose=[] #unrecified camera to GPU/IMU matrix 
poses=[] #GPU/IMU to world transform matrix
R_rectMat=np.eye(4)#recification matrix
pcMat=[]#point cloud matrix
pcMatC=[]#point cloud color matrix
frame=[]#frame index
proj_points=[] #camera coordinate
U=[]#image coordinate u
V=[]#image coordinate v
D=[]#image coordinate depth

#perspective file
perspective=os.path.join(KITTI360Path,"calibration/perspective.txt")
#calib_cam2pose file
calib_cam2pose=os.path.join(KITTI360Path,"calibration/calib_cam_to_pose.txt")

In [6]:
# K,width,height=CameraPerspective.load_intrinsics(perspective)

In [15]:
#intrinsic matrix and recification matrix load from perspective
def load_intrinsics(intrinsic_file):
    #This function is camera Intrinsic matrix, P_rect_00 and R_rect_00
    with open(intrinsic_file) as f:
        intrinsics = f.read().splitlines()
    for line in intrinsics:
        line = line.split(' ')
        #3x4 intrinsic matrix from perspective.txt
        if line[0] == 'P_rect_00:' :
            IntrinsicMat = [float(x) for x in line[1:]]
            IntrinsicMat = np.reshape(IntrinsicMat, [3,4])
 
        #rectification matrix: eye(4) with the left up 3x3 filled with numbers from perspective.txt
        # elif line[0] == 'R_rect_00:':
        #     # R_rect = np.eye(4) 
        #     R_rectMat[:3,:3] = np.array([float(x) for x in line[1:]]).reshape(3,3)

            
    # print("intrinsic",IntrinsicMat)
    # print("R_rect",R_rectMat)
    # print("cam2pose calib",calic2p)
    return IntrinsicMat#, R_rectMat,# calic2p

In [16]:
IntrinsicMat=load_intrinsics(perspective)#read intrinsic matrix and rectification matrix
IntrinsicMat

array([[552.554261,   0.      , 682.049453,   0.      ],
       [  0.      , 552.554261, 238.769549,   0.      ],
       [  0.      ,   0.      ,   1.      ,   0.      ]])

In [ ]:
# #ply read
# def read_ply(seq,str):
#     """ read XYZ point cloud from filename PLY file """
#     pc_path=os.path.join(KITTI360Path,'data_3d_semantics/2013_05_28_drive_00%s_sync/'%seq,str)
#     # print(pc_path)
#     filelist = os.listdir(pc_path)
#     filelist=sorted(filelist)
#     # print('file',filelist)
#     # print(len(filelist))
#     for i in range(len(filelist)):
#         # print(filelist[i],i)
#         plydata = PlyData.read(os.path.join(pc_path,filelist[i]))
#         # print('plydata',plydata)
#         pc_x = np.expand_dims(np.array(plydata['vertex']['x']),1)
#         pc_y = np.expand_dims(np.array(plydata['vertex']['y']),1)
#         pc_z = np.expand_dims(np.array(plydata['vertex']['z']),1)
#         pc_r = np.expand_dims(np.array(plydata['vertex']['red']),1)#.reshape()
#         pc_g = np.expand_dims(np.array(plydata['vertex']['green']),1)
#         pc_b = np.expand_dims(np.array(plydata['vertex']['blue']),1)
#         # print(pc_x.shape,pc_y.shape,pc_z.shape)
#         pc=np.concatenate((pc_x,pc_y,pc_z),1)
#         # print('pcc',pc)
#         pc=np.transpose(np.concatenate((pc_x,pc_y,pc_z),1))
#         # print('pc',pc)
#         pc_color=np.transpose(np.concatenate((pc_r,pc_g,pc_b),1))
        
#         if i == 0:
#             pc_0=pc
#             pc_c0=pc_color
#             pcMat=pc_0
#             pcMatC=pc_c0
#             # print('pc_0',pc_0)
#             # print('pc_c0',pc_c0)
#         # print(pc_0)
#         pcMat=np.array(np.concatenate([pcMat,pc],axis=1))
            
            
#         # pcMat=np.concatenate([pcMat,pc],axis=0)
#         pcMat_RGB=np.array(np.concatenate([pcMatC,pc_color],axis=1))
#     # print(pc.shape)
#     # pcMat
#     return pcMat,pcMat_RGB

# pcMat,pcMatC=read_ply("07","static")
# print('pcMat',pcMat[:,:100],pcMat[:,:100].shape)

In [19]:
# #ply read
def read_oneply(seq,str):
    """ read XYZ point cloud from filename PLY file """
    pc_path=os.path.join(KITTI360Path,'data_3d_semantics/2013_05_28_drive_00%s_sync/'%seq,str,'002782_002902.ply')
    # print(pc_path)
    # filelist = os.listdir(pc_path)
    # filelist=sorted(filelist)
    # print('file',filelist)
    # print(len(filelist))
    # print(filelist[i],i)
    plydata = PlyData.read(pc_path)
    # print('plydata',plydata)
    pc_x = np.expand_dims(np.array(plydata['vertex']['x']),1)
    pc_y = np.expand_dims(np.array(plydata['vertex']['y']),1)
    pc_z = np.expand_dims(np.array(plydata['vertex']['z']),1)
    pc_r = np.expand_dims(np.array(plydata['vertex']['red']),1)#.reshape()
    pc_g = np.expand_dims(np.array(plydata['vertex']['green']),1)
    pc_b = np.expand_dims(np.array(plydata['vertex']['blue']),1)
    # print(pc_x.shape,pc_y.shape,pc_z.shape)
    pc=np.concatenate((pc_x,pc_y,pc_z),1)
    # print('pcc',pc)
    pc=np.transpose(np.concatenate((pc_x,pc_y,pc_z),1))
    # print('pc',pc)
    pc_color=np.transpose(np.concatenate((pc_r,pc_g,pc_b),1))
    
    
    # pc_0=pc
    # pc_c0=pc_color
    # pcMat=pc_0
    # pcMatC=pc_c0
        # print('pc_0',pc_0)
        # print('pc_c0',pc_c0)
    # print(pc_0)
    # pcMat=np.array(np.concatenate([pcMat,pc],axis=1))
        
        
    # pcMat=np.concatenate([pcMat,pc],axis=0)
    # pcMat_RGB=np.array(np.concatenate([pcMatC,pc_color],axis=1))
    # print(pc.shape)
    # pcMat
    return pc,pc_color

pcMat,pcMatC=read_oneply("07","static")
print('pcMat',pcMat,pcMat.shape)

pcMat [[ 880.9501  881.0583  881.0631 ... 1026.45   1026.42   1026.83  ]
 [3274.7288 3274.838  3274.8547 ... 3474.57   3474.66   3475.52  ]
 [ 115.9397  115.7767  115.669  ...  114.729   114.724   114.727 ]] (3, 2342392)


In [11]:
# pc_path=os.path.join(KITTI360Path,'data_3d_semantics/2013_05_28_drive_0000_sync/static/000372_000610.ply')
# data=read_oneply(pc_path, "static")
# print(data,data.shape)

# pcMat= np.vstack((data['x'], data['y'], data['z']))
# pcMatC= np.vstack((data['red'], data['green'], data['blue']))
# print(pcMat,pcMat.shape,pcMatC,pcMatC.shape)

In [20]:
print('pcMat',pcMat,pcMat.shape)

pcMat [[ 880.9501  881.0583  881.0631 ... 1026.45   1026.42   1026.83  ]
 [3274.7288 3274.838  3274.8547 ... 3474.57   3474.66   3475.52  ]
 [ 115.9397  115.7767  115.669  ...  114.729   114.724   114.727 ]] (3, 2342392)


In [13]:
import sys
print(sys.getsizeof(pcMat)/1024**2)

0.0001068115234375


In [21]:
# T

In [22]:
def world2image(pose_array_list, pcMat, IntrinsicMat):
    #This function is to open poses file 
    #cam2poses file
    
    #input: row of cam0_to_world, 1x16 
    #input: point cloud matrix, 3xN
    # warning: pcMat will be destroyed!
    """
    posespath=os.path.join(KITTI360Path,'data_poses/2013_05_28_drive_00%s_sync/'%seq,"cam0_to_world.txt")
    # print('poses',posespath)
    
    with open(posespath) as f:
        posesdata = f.read().splitlines()
    start_time=time.time()
    for line in posesdata:
        line = line.split(' ')
        # print('line',line)
        # for i in line:
        if '' in line:
            line.remove('')
        frame.append(int(line[0]))
        frameID=len(frame)
        print(frameID)
        pcMat,pcMatC=read_ply("00",'static')
    """
        

        # print('line',line)
        # print('line',line[1:])

        # print('frame',line[0])
        # print('len line',len(line))  
    #所有的R和T的来源，每个点的平移和旋转
    pose=np.array([float(x) for x in pose_array_list]).reshape(4,4)
    print('pose',pose)
    R=pose[:3,:3].reshape(3,3)
    T=pose[:3,3].reshape(3,1)
    #R是3✖3，T是3x1。points最终结果为3x1
    print('R',R,R.shape)
    print('T',T,T.shape)
    # print('R',R,R.shape)
    
    # print(int(pcMat.shape[1]))
    
    #R:3x3, pcMat:3xN, T:扩展为3xN    
    # print('T.shape',(np.repeat(np.expand_dims(T,axis=1),pcMat.shape[1],axis=1)).shape)
    # print('mat shape',pcMat[:,:].shape)
    # print('R shape',(R.T).shape)
    # points_local=np.matmul(R.T,(pcMat[:,:]-np.repeat(np.expand_dims(T,axis=1),pcMat.shape[1],axis=1)))
    # points_local=np.matmul(R.T,(pcMat[:,:]-np.repeat(T,pcMat.shape[1],axis=1)))
    # points_local=np.matmul(R.T,(pcMat[:,:]-np.repeat(T,pcMat.shape[1],axis=1)))
        # points_local.append(points)
    print('pcMat',pcMat[:,:],pcMat[:,:].shape)
    print('T',T[:,:],T[:,:].shape)
    print('transfer',(pcMat[:,:]-T[:,:]),(pcMat[:,:]-T[:,:]).shape)
    points_local=np.matmul(R.T,(pcMat-T))
    print('local points',points_local)    
    #local points = 3xN
    print('3x3 Intrinsic',IntrinsicMat[:3,:3])
    print('Intrinsic',IntrinsicMat)
    proj_points=np.matmul(IntrinsicMat[:3,:3],points_local)
    print('image points',proj_points)
    del pcMat
    # proj_points.append(points_proj)
    proj_points[2,:][proj_points[2,:]==0] = -1e-6
    #depth=proj_points[2,:]
    
    #depth[depth==0]= -1e-6
    
    #print(sys.getsizeof(proj_points))

    #normalized image coordinates
    u = np.round(proj_points[0,:]/np.abs(proj_points[2,:]))#.astype(np.int)
    v = np.round(proj_points[1,:]/np.abs(proj_points[2,:]))#.astype(np.int)
        
        # U.append(u)
        # V.append(v)
        # d.append(depth)
        # end_time=time.time()
        # print('pose',pose)
        # pose=np.array(np.concatenate((pose, np.array([0.,0.,0.,1.]).reshape(1,4))))
        # print('pose',pose)
        # print('time',end_time-start_time)
        # poses.appened(pose)
    # print("poses",poses)
            # calculate u, v depth
    print('u',u,u.shape)
    print('v',v,u.shape)
    print('d',proj_points[2,:])
    return u,v,proj_points[2,:]



In [23]:

pcMat,pcMatC=read_oneply("07","static")
# print('pcMat',pcMat,pcMat.shape)
posespath=os.path.join(KITTI360Path,'data_poses/2013_05_28_drive_0007_sync/',"cam0_to_world.txt")
IntrinsicMat=load_intrinsics(perspective)#read intrinsic matrix and rectification matrix
# print('IntrinsicMat',IntrinsicMat,IntrinsicMat.shape)
poses=np.loadtxt(posespath)    
frame=poses[:,0]
pose_initial=poses[0,1:]

print('inital',pose_initial)
u,v,d=world2image(pose_initial, pcMat, IntrinsicMat)
del pcMat

# print('u',u,u.shape)
# print('v',v,v.shape)
# print('depth',d,d.shape)
# del u,v,d

FileNotFoundError: [Errno 2] No such file or directory: '/media/ren/EXTERNAL_USB/KITTI360_DATASET/data_3d_semantics/2013_05_28_drive_0007_sync/static/011079_011287.ply'

In [90]:
max(u)

441.0

In [45]:
pcMat = None

In [46]:
# def transfer4frame(seq,pcMat):
#     posespath=os.path.join(KITTI360Path,'data_poses/2013_05_28_drive_00%s_sync/'%seq,"cam0_to_world.txt")
#     # print('poses',posespath)
#     frame=[]
#     U=[]
#     V=[]
#     D=[]
#     # with open(posespath) as f:
#     #     posesdata = f.read().splitlines()
#     #     # print(posesdata)
#     # # start_time=time.time()
#     # for line in posesdata:
#     #     line = line.split(' ')
#     #     print('line',line)
#     #     # for i in line:
#     #     if '' in line:
#     #         line.remove('')
#     #     print('l',line)
#     #     frame.append(int(line[0]))
#     #     frameID=len(frame)
#     #     print("frameID",frameID)
#     poses=np.loadtxt(posespath)    
#     frame=poses[:,0]
#     poses=poses[:,1:]
#     print('poses',poses.shape)
#     for i in range(poses.shape[0]): 
  
#         u,v,depth=world2image(poses[i,:], pcMat)
#         frame[i]
#         U.append(u.T)
#         V.append(v.T)
        
#         D.append(depth.T)    

        
        
#     return U,V,D
    

In [47]:
# u,v,depth=transfer4frame('00', pcMat)

In [107]:

def uvdepth_to_image(u, v, depth, image_x=1408, image_y=376):
    # input u: N*1 
    # input v: N*1
    # input depth: N*1
    # return: image image_x*image_y
    
    idx = np.where((u >= 0) & (u < image_x) & (v >= 0) & (v < image_y) & (depth >= 0))
    image = np.zeros((image_x, image_y))
    
    for i in idx:
        x = u[i]
        y = v[i]
        d = depth[i]
        image[x,y] = d

    return image

def uvRGBD_to_image(u, v,  depth, rgb_mat, image_x=1408, image_y=376):
    # input u: N*1
    # input v: N*1
    # input rgb_mat: N*3
    # return: image: image_x*image_y*3
    
    idx = np.where((u >= 0) & (u < image_x) & (v >= 0) & (v < image_y) & (depth >= 0))
    print(idx[0].shape)
    # print(idx)
    image = np.zeros((image_x, image_y, 3))
    image_depth = np.ones((image_x, image_y))*np.inf
    # print('image 0',image)
    print(len(idx[0]))
    for i in idx[0]:
        x = int(u[i])
        y = int(v[i])
        rgb = rgb_mat[i,:]
        d = depth[i]
        if d < image_depth[x,y]:
            image[x, y, :] = rgb
            image_depth[x,y] = d
    return image, image_depth
    
    


In [108]:
image1, image_depth=uvRGBD_to_image(u, v, d, pcMatC.T, image_x=1408, image_y=376)
print(pcMatC.T)
print(77)
print(image1.shape)

cv2.imwrite('frame1.png',image1)

(2278721,)
2278721
[[128 128 128]
 [128 128 128]
 [128 128 128]
 ...
 [ 25  26  27]
 [ 26  26  27]
 [ 18  19  19]]
77
(1408, 376, 3)


True

Now, once we get points on frame, we are going to draw them. Using a huge matrix as a plot graph, set the boundary of image(1408,376), put u,v on the image, but ignore all the depth <0. if u and v are same, choose the smallest depth to save. Drawing graph with 3 channel RGB value.  

画图工作：首先拥有像素坐标系的信息，图像像素u和v，深度信息depth，颜色信息RGB。使用一个巨大的空矩阵，作为图像，像素的大小为图片的大小。接着将3D得到的点放入矩阵中，其中，u和v超过图像边缘的点舍去，深度小于0的点舍去。

In [ ]:
imageMat=np.zeros(1408,376)

TypeError: Cannot interpret '376' as a data type